In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import sys
import os
import math
from data import load_NEMSIS_dask, load_NEMSIS_pandas, chain_join

In [2]:
# data_dir: os.PathLike = "/Volumes/Extreme Pro/datasets/NEMSIS/ASCII/"
# data_dir: os.PathLike = "/media/jbajor/Extreme Pro/datasets/NEMSIS/ASCII/"
data_dir: os.PathLike = "/home/jbajor/Dev/datasets/NEMSIS/ASCII/"


nemsis_subset = {
    "ComputedElements":[
        'PcrKey',
        'NasemsoRegion',
        'Urbanicity',
        'ageinyear',
        'EMSDispatchCenterTimeSec',
        'EMSChuteTimeMin',
        'EMSSceneResponseTimeMin',
        'EMSSceneTimeMin',
        'EMSSceneToPatientTimeMin'
    ],
    "FACTPCRADDITIONALRESPONSEMODE":None,
    "FACTPCRADDITIONALSYMPTOM":None,
    "FACTPCRADDITIONALTRANSPORTMODE":None,
    "FACTPCRARRESTCPRPROVIDED":None,
    "FACTPCRARRESTRESUSCITATION":None,
    "FACTPCRARRESTRHYTHMDESTINATION":None,
    "FACTPCRARRESTROSC":None,
    "FACTPCRARRESTWITNESS":None,
    "FACTPCRBARRIERTOCARE":None,
    "FACTPCRMEDICATION":None,
    "FACTPCRPRIMARYIMPRESSION":None,
    "FACTPCRPROCEDURE":None,
    "FACTPCRPROTOCOL":None,
    "FACTPCRSECONDARYIMPRESSION":None,
    "FACTPCRTRAUMACRITERIA":None,
    "FACTPCRWORKRELATEDEXPOSURE":None,
    # "PCRMEDCOMPGROUP":None, # bad key
    # "PCRPROCCOMPGROUP":None, # bad key
    # "PCRVITALECGGROUP":None, # bad key
    "Pub_PCRevents":[
        'PcrKey',
        'eArrest_01',
        'eArrest_02',
        'eArrest_05',
        'eArrest_07',
        'eArrest_11',
        'eArrest_14',
        'eArrest_16',
        'eArrest_18',
        'eDisposition_23',
        'ePayment_01',
        'eResponse_07',
        'eResponse_15',
        'eSituation_01',
        'eSituation_07',
        'eSituation_08',
        'eTimes_01',
        'eTimes_03',
        'eTimes_05',
        'eTimes_06',
        'eTimes_07',
        'eTimes_09',
        'eTimes_11',
        'eTimes_12',
        'eTimes_13'
    ],
}


In [4]:
dataframes = load_NEMSIS_pandas(data_dir=data_dir, load_targets=nemsis_subset, nrows=50000)

In [5]:
dataframes['ComputedElements']

,PcrKey,NasemsoRegion,Urbanicity,ageinyear,EMSDispatchCenterTimeSec,EMSChuteTimeMin,EMSSceneResponseTimeMin,EMSSceneTimeMin,EMSSceneToPatientTimeMin
0,3442467,South,,.,.,.,.,.,.
1,9059962,South,Urban,.,.,.,.,.,.
2,9778729,West,Rural,90,60,2,13,12,.
3,9829108,West,Urban,85,10,4.8333333333,5,60,.
4,10094816,South,Urban,82,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...
499995,29400036,South,Urban,94,.,0,11.633333333,55.6,21.983333333
499996,29400037,South,Urban,80,.,0.0166666667,25.616666667,24.366666667,0.0166666667
499997,29400038,South,Urban,93,.,8.0166666667,10.866666667,39.45,10.3
499998,29400039,South,Urban,.,.,2.2166666667,1.9833333333,.,5.45


In [8]:
key='PcrKey'
data_dir=data_dir
load_targets=nemsis_subset

data_dict = {}
l_iter = iter(load_targets.keys())

In [60]:
f = next(l_iter)
print(f)

Pub_PCRevents


In [64]:
# f = "FACTPCRPROTOCOL"

if f == "FACTPCRMEDICATION":
    data = dd.read_csv(os.path.join(data_dir, f + ".txt"), delimiter="~\|~", engine="python", blocksize=85e6, sample=25000000, dtype={"'eMedications_10'": 'object', "'eMedications_03'": 'object'})

if f == "FACTPCRPROCEDURE":
    data = dd.read_csv(os.path.join(data_dir, f + ".txt"), delimiter="~\|~", engine="python", blocksize=85e6, sample=25000000, dtype={"'eProcedures_10'": 'object', "'eProcedures_03'": 'object'})

if f == "FACTPCRPROTOCOL":
    data = dd.read_csv(os.path.join(data_dir, f + ".txt"), delimiter="~\|~", engine="python", blocksize=85e6, sample=25000000, dtype={"'eProtocol_01'": 'object'})

if f == "Pub_PCRevents":
    data = dd.read_csv(os.path.join(data_dir, f + ".txt"), delimiter="~\|~", engine="python", blocksize=85e6, sample=25000000, dtype={"'eDisposition_23'": 'object', "'eResponse_07'":'object', "'eArrest_11'":'object'})

else:
    data = dd.read_csv(os.path.join(data_dir, f + ".txt"), delimiter="~\|~", engine="python", blocksize=85e6, sample=25000000)

# Remove quotes from headers
data.columns = [col.strip("'") for col in data.columns]

if load_targets[f] is not None:
    data = data[load_targets[f]]

data.set_index(key)

data_dict[f] = data

In [51]:
data_dict['FACTPCRSECONDARYIMPRESSION'].head()

,PcrKey,eSituation_12
0,3442467,NaN
1,9778729,NaN
2,9829108,NaN
3,10094816,NaN
4,10768573,S89.90XA


In [65]:
data_dict.keys()

dict_keys(['ComputedElements', 'FACTPCRADDITIONALRESPONSEMODE', 'FACTPCRADDITIONALSYMPTOM', 'FACTPCRADDITIONALTRANSPORTMODE', 'FACTPCRARRESTCPRPROVIDED', 'FACTPCRARRESTRESUSCITATION', 'FACTPCRARRESTRHYTHMDESTINATION', 'FACTPCRARRESTROSC', 'FACTPCRARRESTWITNESS', 'FACTPCRBARRIERTOCARE', 'FACTPCRPRIMARYIMPRESSION', 'FACTPCRMEDICATION', 'FACTPCRPROCEDURE', 'FACTPCRPROTOCOL', 'FACTPCRSECONDARYIMPRESSION', 'FACTPCRTRAUMACRITERIA', 'FACTPCRWORKRELATEDEXPOSURE', 'Pub_PCRevents'])

In [77]:
for i in data_dict.keys():
    data_dict[i] = data_dict[i].drop(columns=['PcrKey'])

In [78]:
main = data_dict["Pub_PCRevents"]
to_join = [data_dict[i] for i in data_dict.keys() if i != "Pub_PCRevents"]

In [79]:
out = main.join(to_join, how='left')

In [81]:
out.to_csv('/home/jbajor/Desktop/dabp_data/data_*.csv', compute=True)